In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

  Activating project at `D:\Science\Code\Julia\envr`


In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [2]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [3]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [4]:
t = 500.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [5]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [6]:
u0 = SA[17.6055, 0.6181, 0.429232]

3-element SVector{3, Float64} with indices SOneTo(3):
 17.6055
  0.6181
  0.429232

In [12]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201)

0.3:0.00085:0.47

In [13]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [9]:
function init_ds(U0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0range[1]]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save("LSs_200x200.jld", "data", Λs)
    save("u0s_200x200.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [10]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    output(idx_U0, U0_, u0_lc) # Вывод
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ΛΛ = spectrum(ds) # Вычисление спектра
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    output_end_iter(ΛΛ, u0_lc) # Вывод
    save_tofile()
    separate()
    
end

Индекс U0: 1; U0: 0.3
Начальные условия: [17.6055, 0.6181, 0.429232]
Спектр: [0.008525102862872488, -2.7656506319973855, -2.785387286524891]
Последняя точка: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]

---------
Индекс U0: 2; U0: 0.3017
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [-0.0011247963007345577, -2.7760081701896464, -2.790323533807625]
Последняя точка: [14.268328018799107, 0.6029006088767083, 0.42467526109470544]

---------
Индекс U0: 3; U0: 0.3034
Начальные условия: [14.268328018799107, 0.6029006088767083, 0.42467526109470544]
Спектр: [0.0018617457443857817, -2.784870248744019, -2.8038434594603845]
Последняя точка: [4.274536207547047, 0.790942770937852, 0.417786327754446]

---------
Индекс U0: 4; U0: 0.3051
Начальные условия: [4.274536207547047, 0.790942770937852, 0.417786327754446]
Спектр: [0.008851023238294781, -2.7990610207336157, -2.8142497082790277]
Последняя точка: [11.837228229451274, 0.6153428690271009, 0.4


---------
Индекс U0: 32; U0: 0.3527
Начальные условия: [11.757938356862983, 0.5980059543857033, 0.4090796440664437]
Спектр: [0.009117745936964381, -1.8334875166296545, -4.270431982934809]
Последняя точка: [19.670863777456947, 0.6589553231040843, 0.4188957587490039]

---------
Индекс U0: 33; U0: 0.3544
Начальные условия: [19.670863777456947, 0.6589553231040843, 0.4188957587490039]
Спектр: [0.004895598444796603, -1.862776353563096, -4.251986075848624]
Последняя точка: [3.757573594228641, 0.8212910265006373, 0.40831079243930557]

---------
Индекс U0: 34; U0: 0.3561
Начальные условия: [3.757573594228641, 0.8212910265006373, 0.40831079243930557]
Спектр: [0.00940585418497136, -1.9087630743946475, -4.218484888040946]
Последняя точка: [11.598655421894481, 0.5986194608412735, 0.40763588052678634]

---------
Индекс U0: 35; U0: 0.3578
Начальные условия: [11.598655421894481, 0.5986194608412735, 0.40763588052678634]
Спектр: [0.010851531256404442, -1.9406879849056173, -4.199146830631325]
Последняя 


---------
Индекс U0: 63; U0: 0.4054
Начальные условия: [17.013805628223224, 0.7061536811187001, 0.40590568682017975]
Спектр: [0.005080738386288186, -3.115428364344138, -3.1297682844180694]
Последняя точка: [4.3586148358673595, 0.7693969219173014, 0.3897933269509992]

---------
Индекс U0: 64; U0: 0.4071
Начальные условия: [4.3586148358673595, 0.7693969219173014, 0.3897933269509992]
Спектр: [0.01133119518603355, -3.1159916778968086, -3.1285447978557825]
Последняя точка: [4.752147895961934, 0.7513269485760701, 0.38860393418078015]

---------
Индекс U0: 65; U0: 0.4088
Начальные условия: [4.752147895961934, 0.7513269485760701, 0.38860393418078015]
Спектр: [0.006075815137039758, -3.1133312224956073, -3.125661738068189]
Последняя точка: [6.396871005598906, 0.8183280270054508, 0.4008686517545547]

---------
Индекс U0: 66; U0: 0.4105
Начальные условия: [6.396871005598906, 0.8183280270054508, 0.4008686517545547]
Спектр: [0.009812767421244747, -3.1154554779112877, -3.125179128928509]
Последняя т


---------
Индекс U0: 94; U0: 0.4581
Начальные условия: [11.196064787650892, 0.6403562721597423, 0.3752060760045731]
Спектр: [0.004893963032771009, -1.6656969809126088, -4.390508037062765]
Последняя точка: [14.327937822060598, 0.6331166163908192, 0.37784160698805064]

---------
Индекс U0: 95; U0: 0.4598
Начальные условия: [14.327937822060598, 0.6331166163908192, 0.37784160698805064]
Спектр: [0.006082283852465868, -1.320553567360163, -4.740433272237736]
Последняя точка: [8.764501097125802, 0.77198426712923, 0.37968932281777656]

---------
Индекс U0: 96; U0: 0.4615
Начальные условия: [8.764501097125802, 0.77198426712923, 0.37968932281777656]
Спектр: [0.005687988250396953, -1.032493817684293, -5.036205936132197]
Последняя точка: [10.829480167020712, 0.6552227413887237, 0.3720255465795459]

---------
Индекс U0: 97; U0: 0.4632
Начальные условия: [10.829480167020712, 0.6552227413887237, 0.3720255465795459]
Спектр: [0.006785998436743168, -0.7799068690510537, -5.297846498459185]
Последняя точк

In [11]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1
            continue
        end
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ΛΛ = spectrum(ds) # Вычисление спектра
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        output_end_iter(ΛΛ, u0_lc)
        save_tofile()
        separate()
    end
end

Индекс I0: 2; I0: -1.40309
Индекс U0: 1; U0: 0.3
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [-0.004526822603189085, -2.7463575143623196, -2.7638277010757135]
Последняя точка: [17.833951690954507, 0.6231547041048856, 0.42983188477570805]

---------
Индекс I0: 3; I0: -1.40618
Индекс U0: 1; U0: 0.3
Начальные условия: [17.833951690954507, 0.6231547041048856, 0.42983188477570805]
Спектр: [0.004891946451429589, -2.73602982725532, -2.751656755605723]
Последняя точка: [9.14552736655901, 0.6512423889074098, 0.4198397224818873]

---------
Индекс I0: 4; I0: -1.40927
Индекс U0: 1; U0: 0.3
Начальные условия: [9.14552736655901, 0.6512423889074098, 0.4198397224818873]
Спектр: [0.008034914518904838, -2.722777822493918, -2.733648297011648]
Последняя точка: [4.1641264677859455, 0.8161697791822351, 0.42267870940329844]

---------
Индекс I0: 5; I0: -1.41236
Индекс U0: 1; U0: 0.3
Начальные условия: [4.1641264677859455, 0.8161697791822351, 0.42267870940329844]
Сп

Спектр: [0.004418463265079412, -0.12487020089356451, -4.473972376210387]
Последняя точка: [7.00574811787732, 0.6894933906783912, 0.4204662724491416]

---------
Индекс I0: 31; I0: -1.4927
Индекс U0: 1; U0: 0.3
Начальные условия: [7.00574811787732, 0.6894933906783912, 0.4204662724491416]
Спектр: [0.008554211598186815, -0.06816005103262789, -4.500888351913164]
Последняя точка: [4.106306002409835, 0.7951065750375479, 0.42224462920285033]

---------
Индекс I0: 32; I0: -1.49579
Индекс U0: 1; U0: 0.3
Начальные условия: [4.106306002409835, 0.7951065750375479, 0.42224462920285033]
Спектр: [0.009983783922716453, -0.006870188168179773, -4.52961656465231]
Последняя точка: [12.455021341417055, 0.7581048156427137, 0.4378394863206858]

---------
Индекс I0: 33; I0: -1.49888
Индекс U0: 1; U0: 0.3
Начальные условия: [12.455021341417055, 0.7581048156427137, 0.4378394863206858]
Спектр: [0.005680796604544064, -0.11167066501047523, -4.407363862613286]
Последняя точка: [4.8636443763766115, 0.7558405809022188

Последняя точка: [2.700382940562621, 0.85453968100517, 0.43072572384607993]

---------
Индекс I0: 59; I0: -1.57922
Индекс U0: 1; U0: 0.3
Начальные условия: [2.700382940562621, 0.85453968100517, 0.43072572384607993]
Спектр: [0.3132581930387612, 0.0018153333463523658, -4.281695175447437]
Последняя точка: [2.8312672652713093, 0.8388488642032649, 0.4265433199066694]

---------
Индекс I0: 60; I0: -1.58231
Индекс U0: 1; U0: 0.3
Начальные условия: [2.8312672652713093, 0.8388488642032649, 0.4265433199066694]
Спектр: [0.4381620716647167, 0.0002646538127262704, -4.379308358038692]
Последняя точка: [6.310548862719596, 0.7641236065852536, 0.43022469092865273]

---------
Индекс I0: 61; I0: -1.5854
Индекс U0: 1; U0: 0.3
Начальные условия: [6.310548862719596, 0.7641236065852536, 0.43022469092865273]
Спектр: [0.4809275902518324, -0.00225042341643161, -4.358004017346378]
Последняя точка: [9.705424600401555, 0.7451383211544256, 0.43262573404262306]

---------
Индекс I0: 62; I0: -1.58849
Индекс U0: 1; U0


---------
Индекс I0: 87; I0: -1.66574
Индекс U0: 1; U0: 0.3
Начальные условия: [13.476166686232519, 0.652514531471973, 0.44047506917479506]
Спектр: [0.007742698433385432, -0.6258491670364666, -5.127661942038242]
Последняя точка: [13.177634963653087, 0.5469734378734834, 0.46149030470397623]

---------
Индекс I0: 88; I0: -1.66883
Индекс U0: 1; U0: 0.3
Начальные условия: [13.177634963653087, 0.5469734378734834, 0.46149030470397623]
Спектр: [0.0075906044036064685, -0.5862220431321772, -5.163015801543967]
Последняя точка: [6.59102139144398, 0.7373447726824666, 0.4246020934927053]

---------
Индекс I0: 89; I0: -1.67192
Индекс U0: 1; U0: 0.3
Начальные условия: [6.59102139144398, 0.7373447726824666, 0.4246020934927053]
Спектр: [0.011346863337555546, -0.5359282089987073, -5.206507329921976]
Последняя точка: [11.85171197395798, 0.6804751326509116, 0.44531095732086856]

---------
Индекс I0: 90; I0: -1.67501
Индекс U0: 1; U0: 0.3
Начальные условия: [11.85171197395798, 0.6804751326509116, 0.445310

Индекс U0: 2; U0: 0.3017
Начальные условия: [4.7166981283140395, 0.7653197837581138, 0.4178357261655577]
Спектр: [0.01156445232820547, -1.2698019598187684, -3.855457957813389]
Последняя точка: [6.95418140158891, 0.8153474393445431, 0.43194249761822184]

---------
Индекс I0: 16; I0: -1.44635
Индекс U0: 2; U0: 0.3017
Начальные условия: [6.95418140158891, 0.8153474393445431, 0.43194249761822184]
Спектр: [0.008019651490208756, -1.1610071042489776, -3.931785540938276]
Последняя точка: [8.331652329390424, 0.6617573353982009, 0.4199039374651327]

---------
Индекс I0: 17; I0: -1.44944
Индекс U0: 2; U0: 0.3017
Начальные условия: [8.331652329390424, 0.6617573353982009, 0.4199039374651327]
Спектр: [0.007298650374843243, -1.0615620118302165, -3.995744481668217]
Последняя точка: [13.663646531231292, 0.7414147172960601, 0.43521144619687485]

---------
Индекс I0: 18; I0: -1.45253
Индекс U0: 2; U0: 0.3017
Начальные условия: [13.663646531231292, 0.7414147172960601, 0.43521144619687485]
Спектр: [0.00968

...


---------
Индекс I0: 7; I0: -1.41854
Индекс U0: 6; U0: 0.3085
Начальные условия: [11.812816230180209, 0.6119783841055529, 0.4208257687219385]
Спектр: [0.011207069509069904, -1.8337771025971235, -3.6351032341931058]
Последняя точка: [6.860997240868179, 0.693636514046117, 0.41536301277883214]

---------
Индекс I0: 8; I0: -1.42163
Индекс U0: 6; U0: 0.3085
Начальные условия: [6.860997240868179, 0.693636514046117, 0.41536301277883214]
Спектр: [0.012534132175487283, -1.687706969350156, -3.7500917801609877]
Последняя точка: [4.183123264032339, 0.8254297525511006, 0.4227000180388709]

---------
Индекс I0: 9; I0: -1.42472
Индекс U0: 6; U0: 0.3085
Начальные условия: [4.183123264032339, 0.8254297525511006, 0.4227000180388709]
Спектр: [0.012939426962185844, -1.5497404549670892, -3.855877282678298]
Последняя точка: [12.194203678677024, 0.6075768201949996, 0.4215678886545622]

---------
Индекс I0: 10; I0: -1.42781
Индекс U0: 6; U0: 0.3085
Начальные условия: [12.194203678677024, 0.6075768201949996, 


---------
Индекс I0: 35; I0: -1.50506
Индекс U0: 6; U0: 0.3085
Начальные условия: [7.158691046164717, 0.8308974764259336, 0.43691682848181074]
Спектр: [0.011460659845468365, -1.6190281571137106, -3.0939072364333486]
Последняя точка: [5.744462225352509, 0.7244523231754377, 0.4202283690795762]

---------
Индекс I0: 36; I0: -1.50815
Индекс U0: 6; U0: 0.3085
Начальные условия: [5.744462225352509, 0.7244523231754377, 0.4202283690795762]
Спектр: [0.011422217745692218, -2.3388071149872105, -2.357563275292876]
Последняя точка: [8.77777233870467, 0.6366958498482029, 0.424472453726542]

---------
Индекс I0: 37; I0: -1.51124
Индекс U0: 6; U0: 0.3085
Начальные условия: [8.77777233870467, 0.6366958498482029, 0.424472453726542]
Спектр: [0.005460479183401695, -2.3266737041538925, -2.3517171360676548]
Последняя точка: [4.338433917571325, 0.7712987221186919, 0.41566537125201164]

---------
Индекс I0: 38; I0: -1.51433
Индекс U0: 6; U0: 0.3085
Начальные условия: [4.338433917571325, 0.7712987221186919, 0


---------
Индекс I0: 63; I0: -1.59158
Индекс U0: 6; U0: 0.3085
Начальные условия: [2.8057414335608875, 0.8290192679208807, 0.4225705394420804]
Спектр: [0.8261172506018178, 0.001687743373030691, -4.873019977684796]
Последняя точка: [6.587655349567281, 0.6920288016379065, 0.4224341058686608]

---------
Индекс I0: 64; I0: -1.59467
Индекс U0: 6; U0: 0.3085
Начальные условия: [6.587655349567281, 0.6920288016379065, 0.4224341058686608]
Спектр: [0.01642937426692794, -0.308571978532435, -3.8668396312530118]
Последняя точка: [8.02736829557665, 0.7361644421243491, 0.4302393735186189]

---------
Индекс I0: 65; I0: -1.59776
Индекс U0: 6; U0: 0.3085
Начальные условия: [8.02736829557665, 0.7361644421243491, 0.4302393735186189]
Спектр: [0.7202450725564005, -0.005013176685440571, -4.7732373026525785]
Последняя точка: [5.636421711257356, 0.7885844440407868, 0.4281736381503308]

---------
Индекс I0: 66; I0: -1.60085
Индекс U0: 6; U0: 0.3085
Начальные условия: [5.636421711257356, 0.7885844440407868, 0.4


---------
Индекс I0: 91; I0: -1.6781
Индекс U0: 6; U0: 0.3085
Начальные условия: [12.921094026070366, 0.6321434330642978, 0.4319361969110059]
Спектр: [0.008397789336902982, -1.5422837923993904, -4.804686097746708]
Последняя точка: [11.184209507253591, 0.6850821155641885, 0.4417779830192644]

---------
Индекс I0: 92; I0: -1.68119
Индекс U0: 6; U0: 0.3085
Начальные условия: [11.184209507253591, 0.6850821155641885, 0.4417779830192644]
Спектр: [-0.003670894727631647, -1.5295234340974972, -4.840678379591721]
Последняя точка: [4.074744150888042, 0.7693174033221899, 0.41675938008369173]

---------
Индекс I0: 93; I0: -1.68428
Индекс U0: 6; U0: 0.3085
Начальные условия: [4.074744150888042, 0.7693174033221899, 0.41675938008369173]
Спектр: [0.009142147744773961, -1.5203002240727959, -4.885436983486497]
Последняя точка: [1.6657024312780386, 0.8679529823958297, 0.4267196463865946]

---------
Индекс I0: 94; I0: -1.68737
Индекс U0: 6; U0: 0.3085
Начальные условия: [1.6657024312780386, 0.867952982395

LoadError: InterruptException: